# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [1]:
import xlwings as xw
import numpy as np
import pandas as pd

b1 = xw.Book("себестоимостьА_в1.xlsx")
sheet = b1.sheets["Рецептура"]
#Считываем данные о потреблении ингредиентов из диапазона "G7:O10" 
cons = sheet.range("G7:O10").options(np.array).value
#Считываем данные о цене ингредиентов из диапазона "G14:O14"
price_one = sheet.range("G14:O14").options(np.array).value
cost = cons * price_one
#заменяем любые значения NaN  в массиве cost на 0
cost[np.isnan(cost)] = 0
#axis=1 -суммирование по строкам
cost = np.round(np.sum(cost, axis=1).tolist(), 2)
cost

array([21.48, 16.52, 17.42, 18.08])

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [17]:
sheet.range("T7:T10").options(transpose=True).value = cost
sheet.range("T6").value = "Себестоимость"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [ ]:
sheet.range("T4:T6").api.merge()
sheet.range("T4:T6").color = (255,200,0)
sheet.range("T7:T10").color = (244,244,180)

4. Выполнить 3 с помощью "протягиваемых" формул.

In [ ]:
sheet.range("U7").formula = "=SUMPRODUCT(G7:O7, $G$14:$O$14)"
sheet.range("U7").api.autofill(destination=sheet.range("U7:U10").api, type=xw.constants.AutoFillType.xlFillDefault)

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [5]:
import pandas as pd
reviews = pd.read_csv('reviews_sample.csv', delimiter=',')
recipes = pd.read_csv('recipes_sample.csv', delimiter= ',')
recipes = recipes[['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']]
reviews

,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [6]:
recipes

,id,name,minutes,submitted,description,n_ingredients
0,44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,67664,healthy for them yogurt popsicles,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,38798,i can t believe it s spinach,30,2002-08-29,"these were so go, it surprised even me.",8.0
3,35173,italian gut busters,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,84797,love is in the air beef fondue sauces,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...
29995,267661,zurie s holey rustic olive and cheddar bread,80,2007-11-25,this is based on a french recipe but i changed...,10.0
29996,386977,zwetschgenkuchen bavarian plum cake,240,2009-08-24,"this is a traditional fresh plum cake, thought...",11.0
29997,103312,zwiebelkuchen southwest german onion cake,75,2004-11-03,this is a traditional late summer early fall s...,NaN
29998,486161,zydeco soup,60,2012-08-29,this is a delicious soup that i originally fou...,NaN


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [3]:
import pandas as pd
reviews = pd.read_csv('reviews_sample.csv', index_col=0, encoding='utf-8')
recipes = pd.read_csv('recipes_sample.csv', index_col=0, encoding='utf-8')

# Выбор случайных 5% строк из каждой таблицы
reviews_ex = reviews.sample(frac=0.05)
recipes_ex = recipes.sample(frac=0.05)

# Создание объекта ExcelWriter для записи в файл
writer = pd.ExcelWriter('recipes.xlsx')

# Запись таблиц на разные листы в файл
reviews_ex.to_excel(writer, sheet_name='Отзывы', index=False)
recipes_ex.to_excel(writer, sheet_name='Рецепты', index=False)

# Сохранение файла
writer.save()

# Чтение сохраненных данных из файла Excel и вывод на экран
df_reviews = pd.read_excel('recipes.xlsx', sheet_name='Отзывы')
df_recipes = pd.read_excel('recipes.xlsx', sheet_name='Рецепты')

print("Таблица Отзывы:\n", df_reviews)
print("Таблица Рецепты:\n", df_recipes)

Таблица Отзывы:
       user_id  recipe_id        date  rating  \
0      371761      34998  2007-01-29       4   
1      242729     301043  2009-05-21       5   
2      284897     296092  2008-08-03       4   
3       37449     382098  2010-05-22       5   
4      237655      55822  2006-05-03       4   
...       ...        ...         ...     ...   
6330   719313     216294  2010-04-23       5   
6331  1510609     341050  2010-11-25       5   
6332  1161678     311202  2009-03-09       3   
6333   569951      44888  2009-07-20       5   
6334   195444      88233  2007-02-09       5   

                                                 review  
0     I thought this recipe made a really nice chang...  
1     This is just like I grill my lamb chops and al...  
2     Lovely recipe. The family loved it. I wasnt su...  
3     I enjoyed this with some rice. I preferred it ...  
4     I used white whole wheat flour as I ran out of...  
...                                                 ...  


In [1]:
import pandas

reviews = pandas.read_csv('reviews_sample.csv', index_col=0, encoding='utf-8')
recipes = pandas.read_csv('recipes_sample.csv', index_col=0, encoding='utf-8')

reviews_ex = reviews.sample(frac=0.05)
recipes_ex = recipes.sample(frac=0.05)

write_f = pandas.ExcelWriter('recipes.xlsx')

# Запись таблиц на разные листы в файл
reviews_ex.to_excel(write_f, sheet_name='Отзывы', index=False)
recipes_ex.to_excel(write_f, sheet_name='Рецепты', index=False)

# Сохранение файла
write_f.save()

# Чтение сохраненных данных из файла Excel и последующий вывод на экран
df_reviews = pandas.read_excel('recipes.xlsx', sheet_name='Отзывы')
df_recipes = pandas.read_excel('recipes.xlsx', sheet_name='Рецепты')
print("Таблица Рецепты:\n", df_recipes)
print("Таблица Отзывы:\n", df_reviews)

Таблица Рецепты:
           id  minutes  contributor_id   submitted  n_steps  \
0     170753       50          175492  2006-05-30     16.0   
1      94105       15          146731  2004-06-23      4.0   
2     457270        5          157167  2011-05-25      9.0   
3     247667        5          327115  2007-08-20      4.0   
4      41694       30           47559  2002-10-01     16.0   
...      ...      ...             ...         ...      ...   
1495  400953        4         1431918  2009-11-23      NaN   
1496  289466       70          264017  2008-03-01     11.0   
1497  257334       60           37305  2007-10-06      7.0   
1498  321271        4          559493  2008-08-25      5.0   
1499   92813       20          118163  2004-06-07      NaN   

                                            description  n_ingredients  
0     it's okay to lick your fingers! these melt in ...           14.0  
1                    a different kind of chicken salad.            6.0  
2     a refreshing

In [15]:
df_recipes

,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,254410,105,379822,2007-09-20,NaN,i wanted something special for my husband when...,NaN
1,185802,45,198129,2006-09-13,12.0,"after many attempts to perfect my own recipe ,...",13.0
2,77307,70,89831,2003-11-25,12.0,the addition of the pudding mix is what makes ...,13.0
3,204306,20,424680,2007-01-08,8.0,having tried a number of cranberry sauce recip...,6.0
4,161883,30,67728,2006-03-28,NaN,"something a little different, adapted from a l...",16.0
...,...,...,...,...,...,...,...
1495,185486,30,163112,2006-09-11,9.0,onions and leeks have been known in egypt sinc...,NaN
1496,92036,45,22015,2004-05-27,16.0,this is my own creation based upon grandma's g...,8.0
1497,412908,20,347873,2010-02-14,9.0,another mark bittman recipe that i am posting ...,NaN
1498,315470,30,283251,2008-07-24,13.0,everyday with rachael ray,10.0


In [16]:
df_reviews

,user_id,recipe_id,date,rating,review
0,469534,36767,2007-07-16,5,I decided to use orzo pasta tossed with olive ...
1,383206,143845,2009-03-16,4,"While it lacked a certain 'pizzazz' for me, my..."
2,402135,222218,2008-09-16,5,I made a slightly easier version using a tin o...
3,1800294325,202183,2013-11-22,4,Here&#039;s a tip: You can buy a stove top Esp...
4,705251,355242,2009-12-08,4,This is a tasty bread. I wish there was a bit...
...,...,...,...,...,...
6330,266710,78507,2011-01-26,5,LOVED this sandwich!!! Definitely good enough ...
6331,424680,209323,2008-06-02,5,"You've got a great smoothie recipe here, & tha..."
6332,856874,281791,2012-07-26,3,I made this exactly as specified and it was ve...
6333,827921,163713,2009-09-10,5,Ahhhh... just like my mom makes (minus an ingr...


3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
import xlwings as xw
import pandas as pd

# Чтение данных из CSV-файла
recipes_sample = pd.read_csv('recipes_sample.csv', index_col=0, encoding='utf-8')

# Вычисление значения столбца 'seconds_assign'
recipes_sample['seconds_assign'] = recipes_sample['minutes'] * 60

# Создание объекта Book
recipes_ex = xw.Book('recipes.xlsx')

# Запись данных в лист 'Рецепты' в файле 'recipes.xlsx'
recipes_sheet = recipes_ex.sheets['Рецепты']
recipes_sheet.range('H1').value = 'seconds_assign'
recipes_sheet.range('H2').options(index=False).value = recipes_sample['seconds_assign']

# Чтение данных из листа 'Рецепты' в файле 'recipes.xlsx' и вывод на экран
values = recipes_sheet.range('H2:H1501').value
print(values)


4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
import xlwings as xw

wb = xw.Book('recipes.xlsx')
ws = wb.sheets['Рецепты']
ws.range('I1').value = 'seconds_formula'
ws.range('I2').formula = '=TIMEVALUE("00:"&B2)*86400'
last_row = ws.range('A1').current_region.last_cell.row
ws.range('I2:I{}'.format(last_row)).formula = '=TIMEVALUE("00:"&B2)*86400'
ws.range('I2:I{}'.format(last_row)).api.Copy()
ws.range('I3:I{}'.format(last_row)).api.PasteSpecial(-4122)
ws.range('I2').api.CutCopyMode = False
wb.save()

In [ ]:
import xlwings as xw

wb = xw.Book('recipes.xlsx')
ws = wb.sheets['Рецепты']
ws.range('I1').value = 'seconds_formula'
ws.range('I2').formula = '=TIMEVALUE("00:"&B2)*86400'
last_row = ws.range('A1').current_region.last_cell.row
ws.range('I2:I{}'.format(last_row)).formula = '=TIMEVALUE("00:"&B2)*86400'
ws.range('I2:I{}'.format(last_row)).api.Copy()
ws.range('I3:I{}'.format(last_row)).api.PasteSpecial(-4122)
ws.range('I2').api.CutCopyMode = False
wb.save()


In [ ]:
import xlwings as xw
import pandas as pd

# Чтение данных из CSV-файла
recipes_sample = pd.read_csv('recipes_sample.csv', index_col=0, encoding='utf-8')

# Создание объекта Book
answ = xw.Book('recipes.xlsx')

# Вычисление столбца 'seconds_formula' и запись данных в лист 'Рецепты'
ws = wb.sheets['Рецепты']
ws.range('I1').value = 'seconds_formula'
ws.range('I2:I{}'.format(len(recipes_sample)+1)).formula = ['=TIMEVALUE("00:"&B{})*86400'.format(i+2) for i in range(len(recipes_sample))]

# Чтение данных из листа 'Рецепты' в файле 'recipes.xlsx' и вывод на экран
values = ws.range('I2:I{}'.format(len(recipes_sample)+1)).value
print(values)

# Сохранение файла
answ.save()

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
import xlwings as xw

wb = xw.Book('recipes.xlsx')
ws = wb.sheets['Рецепты']

# Получение объектов ячеек H1 и I1
cell_H1 = ws.range('H1')
cell_I1 = ws.range('I1')

# Установка жирного шрифта и центрирование текста для ячеек H1 и I1
cell_H1.api.Font.Bold = True
cell_I1.api.Font.Bold = True
cell_H1.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
cell_I1.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

wb.save()


6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
import xlwings as xw

wb = xw.Book('recipes.xlsx')
ws = wb.sheets['Рецепты']

# Определение значений цветов для разных диапазонов времени приготовления
green = (0, 255, 0)
yellow = (255, 255, 0)
red = (255, 0, 0)

# Получение объекта диапазона ячеек столбца B
minutes_range = ws.range('B2:B1501')

# Изменение цвета ячеек в зависимости от времени приготовления
for cell in minutes_range:
    minutes = cell.value
    if minutes < 5:
        color = green
    elif minutes <= 10:
        color = yellow
    else:
        color = red
    cell.color = color

wb.save()


7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
import xlwings as xw

wb = xw.Book('recipes.xlsx')
ws = wb.sheets['Рецепты']

# Вычисление количества отзывов и запись значения в ячейку I2
ws.range('I1').value = 'n_reviews'
ws.range('I2').formula = '=COUNTIF(C2:C1501, "<>")'

# Расширение формулы на все строки данных и сохранение изменений
last_row = ws.range('C1').current_region.last_cell.row
ws.range('I2').expand('down').formula = '=COUNTIF(C2:C{}, "<>")'.format(last_row)
wb.save()

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
import xlwings as xw

def validate():
    wb = xw.Book('recipes.xlsx')
    reviews_sheet = wb.sheets['Отзывы']
    recipes_sheet = wb.sheets['Рецепты']
    
    # Получение значений оценок и названий рецептов из листа 'Отзывы'
    ratings = reviews_sheet.range('B2:B1501').value
    recipes = reviews_sheet.range('C2:C1501').value
    
    # Проверка валидности оценок и названий рецептов
    for i in range(len(ratings)):
        if not (0 <= ratings[i] <= 5):
            reviews_sheet.range(f'B{i+2}:C{i+2}').color = (255, 0, 0)
        if recipes[i] not in recipes_sheet.range('A2:A1501').value:
            reviews_sheet.range(f'B{i+2}:C{i+2}').color = (255, 0, 0)       
    wb.save()


9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [ ]:
import pandas as pd
import xlwings as xw

# Считываем данные из CSV-файла и сохраняем в DataFrame
data = pd.read_csv('recipes_model.csv', delimiter='\t')

# Создаем новую книгу в Excel
wb = xw.Book()
ws = wb.sheets.add('Модель')

# Записываем данные на лист Модель
ws.range('A1').value = data

# Сохраняем книгу
wb.save('recipes_model.xlsx')

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
import xlwings as xw

# Открываем файл 'recipes_model.xlsx'
wb = xw.Book('recipes_model.xlsx')
ws = wb.sheets['Модель']

# Определяем формулу
formula = '=B2&" "&UPPER(C2)'

# Записываем формулу в ячейку J2 и заполняем ее до последней строки с данными
last_row = ws.range('A' + str(ws.cells.last_cell.row)).end('up').row
ws.range('J2').value = formula
ws.range('J2').api.autofill(ws.range(f'J2:J{last_row}').api, 0)

# Автоматически подгоняем ширину столбца для отображения данных
ws.range('J2:J' + str(last_row)).api.autofit()

# Сохраняем и закрываем файл
wb.save()
wb.close()


11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
import xlwings as xw

# Открываем файл 'recipes_model.xlsx'
wb = xw.Book('recipes_model.xlsx')
sht = wb.sheets['Модель']

# Используем метод options для задания заливки цветом и полужирного шрифта для заголовков таблицы
sht.range('A1:J1').options(bg_color=(0, 204, 255), bold=True).api.Font.Name = 'Calibri'

# Автоматически подгоняем ширину столбцов для отображения данных
sht.autofit('c')

# Добавляем фильтр в заголовок таблицы
sht.range('A1').expand('table').api.AutoFilter()

# Сохраняем и закрываем файл
wb.save()
wb.close()


12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 

In [ ]:
import pandas as pd
import xlwings as xw

# Открываем файл 'recipes_model.xlsx'
wb = xw.Book('recipes_model.xlsx')

# Читаем данные из файла CSV и агрегируем статистику по группам
data = pd.read_csv('recipes_model.csv')
stats = data.groupby('Сущность')['Атрибут'].count()

# Создаем новый лист 'Статистика' и записываем на него статистику
sht = wb.sheets.add('Статистика')
sht.range('A1').value = stats.to_frame()

# Создаем диаграмму на листе 'Статистика'
chart = sht.charts.add()
chart.set_source_data(sht.range('A1').expand())
chart.chart_type = 'column_clustered'
chart.name = 'Статистика'

# Сохраняем и закрываем файл
wb.save()
wb.close()